In [1]:
import sys
sys.path.append('../src')

In [2]:
import pickle
import os
import collections
import numpy as np
import math
import encoder_decoder_uttt as ed
from utictactoe_board import board as c_board
import copy
import torch
import torch.multiprocessing as mp
from alpha_net_uttt import ConnectNet
import datetime
import logging
from tqdm import tqdm

In [3]:
net = ConnectNet()

In [4]:
cuda = torch.cuda.is_available()

In [5]:
net.cuda()

ConnectNet(
  (conv): ConvBlock(
    (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res_0): ResBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res_1): ResBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  

In [6]:
with torch.no_grad():
    current_board = c_board()
    checkmate = False
    board_state = copy.deepcopy(ed.encode_board(current_board))

In [50]:
root = UCTNode(current_board, move=None, parent=DummyNode())

In [51]:
leaf = root.select_leaf()

In [52]:
encoded_s = ed.encode_board(leaf.game)
encoded_s = encoded_s.transpose(2,0,1)
encoded_s = torch.from_numpy(encoded_s).float().cuda()

In [53]:
child_priors, value_estimate = net(encoded_s)

In [54]:
child_priors

tensor([[0.0171, 0.0078, 0.0063, 0.0187, 0.0105, 0.0056, 0.0102, 0.0194, 0.0100,
         0.0164, 0.0176, 0.0144, 0.0229, 0.0050, 0.0216, 0.0104, 0.0180, 0.0100,
         0.0129, 0.0135, 0.0112, 0.0115, 0.0063, 0.0254, 0.0129, 0.0245, 0.0110,
         0.0090, 0.0096, 0.0119, 0.0127, 0.0034, 0.0086, 0.0048, 0.0196, 0.0061,
         0.0235, 0.0122, 0.0101, 0.0095, 0.0135, 0.0134, 0.0101, 0.0056, 0.0174,
         0.0169, 0.0129, 0.0145, 0.0106, 0.0147, 0.0085, 0.0093, 0.0107, 0.0135,
         0.0107, 0.0183, 0.0168, 0.0136, 0.0043, 0.0107, 0.0130, 0.0092, 0.0099,
         0.0152, 0.0123, 0.0115, 0.0090, 0.0090, 0.0077, 0.0191, 0.0056, 0.0084,
         0.0084, 0.0163, 0.0101, 0.0104, 0.0097, 0.0161, 0.0098, 0.0154, 0.0159]],
       device='cuda:0', grad_fn=<ExpBackward>)

In [55]:
value_estimate

tensor([[-0.2636]], device='cuda:0', grad_fn=<TanhBackward>)

In [56]:
child_priors = child_priors.detach().cpu().numpy().reshape(-1)
value_estimate = value_estimate.item()

In [57]:
child_priors

array([0.01705102, 0.00777669, 0.0063078 , 0.01867882, 0.01054201,
       0.00560169, 0.01020111, 0.01935214, 0.0099953 , 0.01636836,
       0.01757073, 0.01436909, 0.02292205, 0.00502619, 0.02155052,
       0.01036641, 0.01796573, 0.00998601, 0.01287861, 0.01351345,
       0.01116304, 0.01148709, 0.00629348, 0.02543472, 0.01290886,
       0.02450569, 0.01101202, 0.00904266, 0.00964045, 0.01190577,
       0.0127175 , 0.00340463, 0.00856017, 0.00483537, 0.01959443,
       0.00614132, 0.02352763, 0.01221078, 0.0100853 , 0.00952635,
       0.01352199, 0.01342555, 0.01011256, 0.00559576, 0.01739274,
       0.01694698, 0.01285821, 0.01454341, 0.01055109, 0.01468434,
       0.00845387, 0.00930174, 0.01065725, 0.01349898, 0.01066831,
       0.01830662, 0.01676542, 0.01358381, 0.00430858, 0.01072281,
       0.01295201, 0.00920067, 0.00989054, 0.01520835, 0.01231836,
       0.01146843, 0.00903359, 0.00904995, 0.00767214, 0.01912075,
       0.00556602, 0.00835548, 0.00842852, 0.01634807, 0.01007

In [39]:
leaf.game.check_winner()

False

In [40]:
leaf.game.actions()

[(6, 6), (6, 7), (6, 8), (7, 6), (7, 7), (7, 8), (8, 6), (8, 7), (8, 8)]

In [41]:
valid_moves = leaf.game.actions()

In [42]:
child_priors[valid_moves]

/home/sf/Applications/anaconda3/envs/torchenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


IndexError: too many indices for array

In [43]:
c_p = child_priors
action_idxs = leaf.game.actions()
c_p[[i for i in range(len(child_priors)) if i not in action_idxs]] = 0.000000000

In [44]:
c_p

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [45]:
child_priors[(6,6)]

IndexError: too many indices for array

In [46]:
leaf.action_idxes

[]

In [48]:
leaf.expand(child_priors)

[60, 61, 62, 69, 70, 71, 78, 79, 80]


AttributeError: 'UCTNode' object has no attribute 'action_idxs'

In [21]:
def return_winner(winner):
    if winner == "O" or winner == "X":
        print(1)
        return True
    elif winner == "tie":
        print(2)
        return winner
    else:
        return False

In [22]:
winner = None

In [20]:
winner == "O" or "X"

'X'

In [23]:
return_winner(winner=None)

False

In [23]:
leaf.game.mega_board

array([[' ', ' ', ' '],
       [' ', ' ', ' '],
       [' ', ' ', ' ']], dtype='<U32')

In [24]:
leaf.game.current_board

array([[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']], dtype='<U32')

In [23]:
y = check_ninebox(leaf.game.mega_board)

In [24]:
y

In [21]:
def check_ninebox(mat):
    """
    This function can be use to check the small nine-box \
            or the large nine-box.
    input: mat: a matrix of 3x3
    """
    # look for rows, columns, diagnols
    for i in range(3):
        if mat[i,0] == mat[i,1] == mat[i,2] != ' ':
            return mat[i,0]
        if mat[0,i] == mat[1,i] == mat[2,i] != ' ':
            return mat[0,i]
    if mat[0,0] == mat[1,1] == mat[2,2] != ' ':
        return mat[0,0]
    if mat[0,2] == mat[1,1] == mat[2,0] != ' ':
        return mat[1,1]

    # look for tie
    count = 0
    for i in range(3):
        for j in range(3):
            if mat[i,j] != ' ':
                count += 1
    if count == 9:
        return 'tie'
    else:
        return None

In [49]:
class UCTNode():
    def __init__(self, game, move, parent=None):
        self.game = game # state s
        self.move = move # action index
        self.is_expanded = False
        self.parent = parent  
        self.children = {}
        self.child_priors = np.zeros([81], dtype=np.float32)
        self.child_total_value = np.zeros([81], dtype=np.float32)
        self.child_number_visits = np.zeros([81], dtype=np.float32)
        self.action_idxes = []
        
    @property
    def number_visits(self):
        return self.parent.child_number_visits[self.move]

    @number_visits.setter
    def number_visits(self, value):
        self.parent.child_number_visits[self.move] = value
    
    @property
    def total_value(self):
        return self.parent.child_total_value[self.move]
    
    @total_value.setter
    def total_value(self, value):
        self.parent.child_total_value[self.move] = value
    
    def child_Q(self):
        return self.child_total_value / (1 + self.child_number_visits)
    
    def child_U(self):
        return math.sqrt(self.number_visits) * (
            abs(self.child_priors) / (1 + self.child_number_visits))
    
    def best_child(self):
        if self.action_idxes != []:
            bestmove = self.child_Q() + self.child_U()
            bestmove = self.action_idxes[np.argmax(bestmove[self.action_idxes])]
        else:
            bestmove = np.argmax(self.child_Q() + self.child_U())
        return bestmove
    
    def select_leaf(self):
        current = self
        while current.is_expanded:
            best_move = current.best_child()
            current = current.maybe_add_child(best_move)
        return current
    
    def add_dirichlet_noise(self,action_idxs,child_priors):
        valid_child_priors = child_priors[action_idxs] # select only legal moves entries in child_priors array
        valid_child_priors = 0.75*valid_child_priors + 0.25*np.random.dirichlet(np.zeros([len(valid_child_priors)], \
                                                                                          dtype=np.float32)+192)
        child_priors[action_idxs] = valid_child_priors
        return child_priors
    
    def expand(self, child_priors):
        self.is_expanded = True
        action_idxs = self.game.actions()
        c_p = child_priors
        if action_idxs == []:
            self.is_expanded = False
        else:
            for i,j in action_idxs:
                self.action_idxes.append(i*9+j)
        print(self.action_idxes)
        #self.action_idxes = action_idxs
        #for i in range(9):
        #    for j in range(9):
        #        if (i,j) not in action_idxs:
        #            c_p[i,j] = 0.00000
        c_p[[i for i in range(len(child_priors)) if i not in self.action_idxs]] = 0.000000000 # mask all illegal actions
        if self.parent.parent == None: # add dirichlet noise to child_priors in root node
            c_p = self.add_dirichlet_noise(self.action_idxes,c_p)
        self.child_priors = c_p
    
    def decode_n_move_pieces(self,board,move):
        board.drop_piece(move)
        return board
            
    def maybe_add_child(self, move):
        if move not in self.children:
            copy_board = copy.deepcopy(self.game) # make copy of board
            copy_board = self.decode_n_move_pieces(copy_board,move)
            self.children[move] = UCTNode(
              copy_board, move, parent=self)
        return self.children[move]
    
    def backup(self, value_estimate: float):
        current = self
        while current.parent is not None:
            current.number_visits += 1
            if current.game.player == 1: # same as current.parent.game.player = 0
                current.total_value += (1*value_estimate) # value estimate +1 = O wins
            elif current.game.player == 0: # same as current.parent.game.player = 1
                current.total_value += (-1*value_estimate)
            current = current.parent

In [8]:
class DummyNode(object):
    def __init__(self):
        self.parent = None
        self.child_total_value = collections.defaultdict(float)
        self.child_number_visits = collections.defaultdict(float)